<a href="https://colab.research.google.com/github/TomonoriSatoh/ai-basics/blob/proposal/230909_simple_chat_with_relational_db_7_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import sqlite3

conn = sqlite3.connect('user_support.db')
cursor = conn.cursor()

cursor.execute('DROP TABLE IF EXISTS users')
cursor.execute('DROP TABLE IF EXISTS products')
cursor.execute('DROP TABLE IF EXISTS order_history')
cursor.execute('DROP TABLE IF EXISTS support_history')

cursor.execute('''
CREATE TABLE users(
  user_id INTEGER PRIMARY KEY,
  first_name TEXT,
  last_name TEXT,
  email TEXT,
  phone TEXT
)
''')

cursor.execute('''
CREATE TABLE products(
  product_id INTEGER PRIMARY KEY,
  product_name TEXT,
  price INTEGER
)
''')

cursor.execute('''
CREATE TABLE order_history(
  history_id INTEGER PRIMARY KEY,
  user_id INTEGER,
  product_id INTEGER,
  purchase_date DATE,
  quantity INTEGER,
  remarks TEXT,
  created_at DATETIME,
  FOREIGN KEY(user_id) REFERENCES users(user_id),
  FOREIGN KEY(product_id) REFERENCES products(product_id)
)
''')

cursor.execute('''
CREATE TABLE support_history(
  history_id INTEGER PRIMARY KEY,
  user_id INTEGER,
  subject TEXT,
  message_content TEXT,
  message_type TEXT,
  status TEXT,
  created_at DATETIME,
  FOREIGN KEY(user_id) REFERENCES user(user_id)
)
''')

conn.commit()
conn.close()

In [57]:
import sqlite3
from datetime import datetime

conn = sqlite3.connect('user_support.db')
cursor = conn.cursor()

users_data = [
    (1, '太郎', '山田', 'taro@example.com', '090-1234-5678'),
    (2, '花子', '佐藤', 'hanako@example.com', '080-9876-5432')
]

for user in users_data:
  cursor.execute('''
  INSERT INTO users (user_id, first_name, last_name, email, phone)
  VALUES (?, ?, ?, ?, ?)
  ''', user)

products_data = [
    (1, '商品A', 1000),
    (2, '商品B', 2000),
    (3, '商品C', 3000)
]

for product in products_data:
  cursor.execute('''
  INSERT INTO products (product_id, product_name, price)
  VALUES (?, ?, ?)
  ''', product)

order_history_data = [
    (1,1,1,'2023-04-01', 2, '迅速な発送', datetime.now()),
    (2,2,2,'2023-04-05', 1, 'ギフトラッピング', datetime.now()),
]

for order in order_history_data:
  cursor.execute('''
  INSERT INTO order_history(history_id, user_id, product_id, purchase_date, quantity, remarks, created_at)
  VALUES (?, ?, ?, ?, ?, ?, ?)
  ''', order)

support_history_data = [
    (1, 1, '請求に関する問題', None, None, 'open', datetime.now()),
    (2, 1, None, '請求に問題があります。', 'user', None, datetime.now()),
    (3, 1, None, 'お問合せいただきありがとうございます。問題を調査しています。', 'support', None, datetime.now())
]

for support in support_history_data:
  cursor.execute('''
  INSERT INTO support_history (history_id, user_id, subject, message_content, message_type, status, created_at)
  VALUES (?, ?, ?, ?, ?, ?, ?)
  ''', support)

conn.commit()
conn.close()

In [58]:
import sqlite3

conn = sqlite3.connect('user_support.db')
cursor = conn.cursor()

tables = ['users', 'products', 'order_history', 'support_history']

for table in tables:
  print(f"{table} テーブルのデータ")
  cursor.execute(f"SELECT * FROM {table}")
  rows = cursor.fetchall()

  for row in rows:
    print(row)

  print()

conn.close()

users テーブルのデータ
(1, '太郎', '山田', 'taro@example.com', '090-1234-5678')
(2, '花子', '佐藤', 'hanako@example.com', '080-9876-5432')

products テーブルのデータ
(1, '商品A', 1000)
(2, '商品B', 2000)
(3, '商品C', 3000)

order_history テーブルのデータ
(1, 1, 1, '2023-04-01', 2, '迅速な発送', '2023-09-10 11:01:51.484306')
(2, 2, 2, '2023-04-05', 1, 'ギフトラッピング', '2023-09-10 11:01:51.484310')

support_history テーブルのデータ
(1, 1, '請求に関する問題', None, None, 'open', '2023-09-10 11:01:51.484657')
(2, 1, None, '請求に問題があります。', 'user', None, '2023-09-10 11:01:51.484660')
(3, 1, None, 'お問合せいただきありがとうございます。問題を調査しています。', 'support', None, '2023-09-10 11:01:51.484661')

